Environment && Library 

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"


import subprocess
with open(os.path.expanduser("~/successful NUMA node read from SysFS had negative value (-1)"), "r") as f:
    password = f.readline()
proc = subprocess.Popen(['sudo', '-S', 'sh', '-c', 'for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,universal_newlines= True)
stdout, stderr = proc.communicate(input=password+'\n')
print(stdout)
print(stderr)

import numpy as np
import tensorflow as tf

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

[sudo] password for user: 


Configuration

In [2]:
augmentation = True
quant = True
if(augmentation == True):
    model = tf.keras.models.load_model(os.path.expanduser("~/Desktop/augmentation_functionAPI/"))
elif(augmentation == False):
    model = tf.keras.models.load_model(os.path.expanduser("~/Desktop/Non_augmentation_functionAPI/"))
model.summary()
print(model.dtype)

2023-07-22 19:39:21.448184: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 19:39:21.757184: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3817 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 C1 (Conv2D)                    (None, 24, 24, 6)    156         ['input[0][0]']                  
                                                                                                  
 S2 (AveragePooling2D)          (None, 12, 12, 6)    0           ['C1[0][0]']                     
                                                                                                  
 C3 (Conv2D)                    (None, 8, 8, 16)     2416        ['S2[0][0]']                     
                                                                                              

Divide Model

In [3]:
trunk_1 = tf.keras.Model(inputs = model.layers[0].input, outputs = model.layers[1].output)      ### Input ~ C1 ###          
branch_1 = tf.keras.Model(inputs = model.layers[7].input, outputs = model.layers[10].output)    ### Flatten_1 ~ Exit_1 ###
trunk_2 = tf.keras.Model(inputs = model.layers[2].input, outputs = model.layers[3].output)      ### S2 ~ C3 ###
branch_2 = tf.keras.Model(inputs = model.layers[8].input, outputs = model.layers[11].output)    ### Flatten_2 ~ Exit_2 ###
trunk_3 = tf.keras.Model(inputs = model.layers[4].input, outputs = model.layers[6].output)      ### S4 ~ F5 ###
branch_3 = tf.keras.Model(inputs = model.layers[12].input, outputs = model.layers[12].output)   ### Exit_3 ###
branch_4 = tf.keras.Model(inputs = model.layers[9].input, outputs = model.layers[13].output)    ### F6 ~ Exit_4 ###

module_name = ["trunk_1", "branch_1", "trunk_2", "branch_2", "trunk_3", "branch_3", "branch_4"]
module_array = [trunk_1, branch_1, trunk_2, branch_2, trunk_3, branch_3, branch_4]

Dataset

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.reshape(-1, 28 ,28, 1).astype(np.float32) / 255.0, x_test.reshape(-1, 28 ,28, 1).astype(np.float32) / 255.0
if(augmentation == True):
    train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range= 30, width_shift_range= 0.2, height_shift_range= 0.2, zoom_range= 0.2, data_format= 'channels_last', dtype= np.float32)
elif(augmentation == False):
    train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range= 0, width_shift_range= 0, height_shift_range= 0, zoom_range= 0, data_format= 'channels_last', dtype= np.float32)

float32

In [5]:
if(quant == False):
    for i in range(7):
        converter = tf.lite.TFLiteConverter.from_keras_model(module_array[i])
        tflite_model = converter.convert()

        # Save the unquantized/float model:
        save_path = os.path.expanduser("~/Desktop/") + module_name[i] + "_float32.tflite"
        with open(save_path, "wb") as f:
            f.write(tflite_model)

uint8

In [6]:
def inference(x, trunk_1, trunk_2, trunk_3, step):
    if(step >= 1):
        interpreter = tf.lite.Interpreter(model_content = trunk_1)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        interpreter.set_tensor(input_details[0]['index'], x)
        interpreter.invoke()
        x = interpreter.get_tensor(output_details[0]['index'])

    if(step >= 3):
        interpreter = tf.lite.Interpreter(model_content = trunk_2)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        interpreter.set_tensor(input_details[0]['index'], x)
        interpreter.invoke()
        x = interpreter.get_tensor(output_details[0]['index'])

    if(step >= 5):
        interpreter = tf.lite.Interpreter(model_content = trunk_3)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        interpreter.set_tensor(input_details[0]['index'], x)
        interpreter.invoke()
        x = interpreter.get_tensor(output_details[0]['index'])
    
    return x

In [7]:
if(quant == True):
    for i in range(7):
        train_gen.fit(x_train)
        train_generator = train_gen.flow(x_train, y_train, batch_size= 1, shuffle=True, seed=2, save_to_dir= None)

        def representative_data_gen():
            x,y = train_generator.next()
            x = inference(x= x, trunk_1= trunk_1, trunk_2= trunk_2, trunk_3= trunk_3, step= i)
                
            for input_value in tf.data.Dataset.from_tensor_slices(x).batch(1).take(100):
                yield [input_value]


        converter = tf.lite.TFLiteConverter.from_keras_model(module_array[i])
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_data_gen
        # Ensure that if any ops can't be quantized, the converter throws an error
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        # Set the input and output tensors to uint8 (APIs added in r2.3)
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8
        module_quant = converter.convert()

        # Save the quantized model:
        save_path = os.path.expanduser("~/Desktop/") + module_name[i] + "_uint8.tflite"
        with open(save_path, "wb") as f:
            f.write(module_quant)
            
        converter = tf.lite.TFLiteConverter.from_keras_model(module_array[i])
        if(i == 0):
            trunk_1 = converter.convert()
        elif(i == 2):
            trunk_2 = converter.convert()
        elif(i == 4):
            trunk_3 = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp94wvsil6/assets


INFO:tensorflow:Assets written to: /tmp/tmp94wvsil6/assets
/home/user/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-07-22 19:39:22.713130: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-07-22 19:39:22.713150: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-07-22 19:39:22.713588: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp94wvsil6
2023-07-22 19:39:22.713958: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-07-22 19:39:22.713973: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmp94wvsil6
2023-07-22 19:39:22.715514: I tensorflow/compiler/mlir/mlir_graph_op

INFO:tensorflow:Assets written to: /tmp/tmp_acftxlz/assets


INFO:tensorflow:Assets written to: /tmp/tmp_acftxlz/assets
2023-07-22 19:39:22.988001: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-07-22 19:39:22.988020: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-07-22 19:39:22.988120: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp_acftxlz
2023-07-22 19:39:22.988523: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-07-22 19:39:22.988535: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmp_acftxlz
2023-07-22 19:39:22.989532: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-07-22 19:39:22.996566: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmp_acftxlz
2023-07-22 19:39:23.000138: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmpydq1kd7v/assets


...

INFO:tensorflow:Assets written to: /tmp/tmp8px7sh3w/assets
2023-07-22 19:39:24.520902: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-07-22 19:39:24.520920: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-07-22 19:39:24.521022: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp8px7sh3w
2023-07-22 19:39:24.521442: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-07-22 19:39:24.521453: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmp8px7sh3w
2023-07-22 19:39:24.522731: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-07-22 19:39:24.529934: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmp8px7sh3w
2023-07-22 19:39:24.533689: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmpz72nhdhh/assets


INFO:tensorflow:Assets written to: /tmp/tmpz72nhdhh/assets
/home/user/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-07-22 19:39:24.732562: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-07-22 19:39:24.732580: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-07-22 19:39:24.732681: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpz72nhdhh
2023-07-22 19:39:24.733047: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-07-22 19:39:24.733059: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpz72nhdhh
2023-07-22 19:39:24.733881: I tensorflow/cc/saved_model/loader.cc:22

INFO:tensorflow:Assets written to: /tmp/tmpdhk4gnft/assets


INFO:tensorflow:Assets written to: /tmp/tmpdhk4gnft/assets
/home/user/.local/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-07-22 19:39:25.018951: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-07-22 19:39:25.018968: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-07-22 19:39:25.019070: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpdhk4gnft
2023-07-22 19:39:25.019468: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-07-22 19:39:25.019479: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpdhk4gnft
2023-07-22 19:39:25.020896: I tensorflow/cc/saved_model/loader.cc:22